##### Datenverarbeitung des Global Population and Migration Dataset
##### Dieses Notebook lädt die heruntergeladenen Daten und bereitet sie für die Analyse auf.
##### 
##### Last Updated: 15.12.2024
##### ===============================================


#### **1. Imports und Datenladen**
#### ------------------------

In [59]:
import pandas as pd
import numpy as np
from pathlib import Path
import numpy as np

##### Daten laden

In [60]:
print("1. Verfügbare Dateien prüfen")
data_path = Path('../data')

print("CSV-Dateien im data-Ordner:")
for file in data_path.glob('*.csv'):
   print(f"- {file.name}")
   # Speichere die Daten in df
   df = pd.read_csv(file)
   
print("\n2. Überblick über die geladenen Daten:")
print(f"Shape des Dataframes: {df.shape}")
print("\nSpalten im Dataframe:")
print(df.columns.tolist())
print("\nErste Zeilen der Daten:")
print(df.head())

1. Verfügbare Dateien prüfen
CSV-Dateien im data-Ordner:
- processed_world_pop_mig_186_countries.csv
- world_pop_mig_186_countries.csv

2. Überblick über die geladenen Daten:
Shape des Dataframes: (11904, 5)

Spalten im Dataframe:
['country', 'year', 'population', 'netMigration', 'population_in_millions']

Erste Zeilen der Daten:
       country  year  population  netMigration  population_in_millions
0  Afghanistan  2023  42239854.0      -65846.0                      42
1  Afghanistan  2022  41128771.0      -65846.0                      41
2  Afghanistan  2021  40099462.0     -183672.0                      40
3  Afghanistan  2020  38972230.0      166821.0                      38
4  Afghanistan  2019  37769499.0       -8082.0                      37


##### Datentypen anpassen

In [61]:
df['netMigration'] = df['netMigration'].astype(int)

#### **2. Länderbasierte Berechnungen**
# ----------------------------

In [62]:
# Aggregation nach Ländern
country_metrics = df.groupby(by="country").agg({
    'population': 'mean',
    'netMigration': 'sum',
    'population_in_millions': 'mean'
}).reset_index()

# Top/Bottom Migration
top_immigration = country_metrics.nlargest(10, 'netMigration')
top_emigration = country_metrics.nsmallest(10, 'netMigration')

#### **3. Zeitbasierte Berechnungen**
# --------------------------

In [63]:
yearly_metrics = df.groupby('year').agg({
    'population': 'sum',
    'netMigration': 'sum',
    'population_in_millions': 'sum'
}).reset_index()

#### **4. Aktuelle Bevölkerungszahlen (2023)**
# ----------------------------------

In [64]:
current_population = df[df['year'] == 2023].copy()
top_population_countries = current_population.nlargest(10, 'population')

#### **5. Migrations-Kategorisierung**
# --------------------------

In [65]:
migration_categories = pd.cut(df['netMigration'], 
                            bins=[float('-inf'), -1, 0, float('inf')],
                            labels=['Immigration', 'Neutral', 'Emigration'])
migration_distribution = migration_categories.value_counts()

#### **6. Regionale Statistiken**
# ----------------------
##### Migrations-Statistiken nach Kontinenten/Regionen (falls verfügbar)

In [67]:
if 'region' in df.columns:
    regional_stats = df.groupby('region').agg({
        'netMigration': ['sum', 'mean'],
        'population': ['mean', 'max']
    }).round(2)

#### **7. Korrelationsanalyse**
# --------------------

In [68]:
correlation_matrix = df.select_dtypes(include=[np.number]).corr()

#### **8. Daten speichern**
# ----------------
##### Speichere alle berechneten Daten für Visualisierung

In [69]:
processed_data = {
    'country_metrics': country_metrics,
    'top_immigration': top_immigration,
    'top_emigration': top_emigration,
    'yearly_metrics': yearly_metrics,
    'top_population': top_population_countries,
    'migration_distribution': migration_distribution,
    'correlation_matrix': correlation_matrix
}

##### Speichere die Daten als CSV

In [70]:
for name, data in processed_data.items():
    if isinstance(data, pd.DataFrame):
        data.to_csv(data_path / f'analyzed_{name}.csv', index=False)
    elif isinstance(data, pd.Series):
        data.to_frame().to_csv(data_path / f'analyzed_{name}.csv')

print("\nDatenverarbeitung abgeschlossen! Alle Kennzahlen wurden berechnet und gespeichert.")


Datenverarbeitung abgeschlossen! Alle Kennzahlen wurden berechnet und gespeichert.


##### **9. Übersicht der Ergebnisse**
# -------------------------

In [71]:
print("\nÜbersicht der wichtigsten Kennzahlen:")
print(f"Anzahl untersuchter Länder: {len(country_metrics)}")
print(f"Untersuchungszeitraum: {yearly_metrics['year'].min()} - {yearly_metrics['year'].max()}")
print("\nTop 3 Immigrationsländer:")
print(top_immigration[['country', 'netMigration']].head(3))
print("\nTop 3 Emigrationsländer:")
print(top_emigration[['country', 'netMigration']].head(3))


Übersicht der wichtigsten Kennzahlen:
Anzahl untersuchter Länder: 186
Untersuchungszeitraum: 1960 - 2023

Top 3 Immigrationsländer:
                country  netMigration
179       United States      67104784
63              Germany      13307249
141  Russian Federation      12488328

Top 3 Emigrationsländer:
        country  netMigration
14   Bangladesh     -20504716
129    Pakistan     -18487914
37        China     -17070233
